In [419]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn

# 현재 작업 디렉토리로 설정 (Jupyter Notebook 호환)
CURRENT_FILE_PATH = os.getcwd()

# TitanicDataset: 학습 및 검증 데이터를 위한 Dataset 클래스 정의
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        # 입력 데이터를 float 타입의 텐서로 변환하여 저장
        self.X = torch.FloatTensor(X)
        # 타겟 레이블 데이터를 long 타입의 텐서로 변환하여 저장 (분류 문제를 위한 정수형)
        self.y = torch.LongTensor(y)

    def __len__(self):
        # 데이터셋의 전체 샘플 개수 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 샘플의 입력 데이터와 타겟 레이블 반환
        feature = self.X[idx]
        target = self.y[idx]
        # 딕셔너리 형태로 반환하여 DataLoader에서 사용 가능
        return {'input': feature, 'target': target}

    def __str__(self):
        # 데이터셋 정보 문자열로 출력 (데이터 크기, 입력, 타겟 형상 포함)
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )
        return str

In [420]:
# TitanicTestDataset: 테스트 데이터를 위한 Dataset 클래스 정의 (타겟이 없기 때문에 input만 필요)
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        # 입력 데이터를 float 타입의 텐서로 변환하여 저장
        self.X = torch.FloatTensor(X)

    def __len__(self):
        # 데이터셋의 전체 샘플 개수 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 샘플의 입력 데이터 반환
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        # 테스트 데이터셋 정보 문자열로 출력 (데이터 크기와 입력 형상 포함)
        str = "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )
        return str

In [421]:
# 데이터셋을 전처리하여 학습, 검증, 테스트 데이터셋으로 분할하는 함수
def get_preprocessed_dataset():
    # 학습 및 테스트 데이터 파일 경로 지정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # 데이터 파일 읽기
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 학습 및 테스트 데이터 결합 (전처리를 위해)
    all_df = pd.concat([train_df, test_df], sort=False)

    # 여러 단계의 전처리 적용
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # 학습 데이터와 타겟 분리
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    # 테스트 데이터 준비
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # TitanicDataset 객체 생성하여 학습, 검증, 테스트 데이터셋으로 분할
    dataset = TitanicDataset(train_X.values, train_y.values)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset

In [422]:
# Pclass별로 평균 Fare 값을 계산하여 결측치를 채우는 전처리 함수
def get_preprocessed_dataset_1(all_df):
    # Pclass(등급)별 평균 Fare 계산
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    # Fare의 결측치를 등급별 평균 Fare 값으로 채움
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

In [423]:
# 이름(Name)을 사용하여 성(family_name), 호칭(honorific), 이름(name)을 분리하는 전처리 함수
def get_preprocessed_dataset_2(all_df):
    # 이름을 쉼표(,)와 점(.)을 기준으로 분리하여 세 개의 컬럼 생성
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    # 각 문자열을 공백 제거하여 정리
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    # 원래 데이터프레임에 분리된 이름 데이터를 추가
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

In [424]:
# honorific(호칭)별로 나이의 중앙값을 계산하여 Age 결측치를 채우는 전처리 함수
def get_preprocessed_dataset_3(all_df):
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    # 호칭별 중앙값으로 Age의 결측치를 채움
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    # 필요 없는 열 제거
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

In [425]:
# 가족 수(family_num)와 혼자 탑승 여부(alone)를 나타내는 컬럼 추가 및 불필요한 열 제거
def get_preprocessed_dataset_4(all_df):
    # 가족 수(family_num) 계산하여 새로운 컬럼에 저장
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    # 가족 수가 0이면 혼자 탑승한 것으로 간주하여 alone 컬럼에 1을 할당
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    # 가족이 있으면 alone 값을 0으로 설정
    all_df["alone"].fillna(0, inplace=True)
    # 필요 없는 열(식별자, 이름 등) 삭제
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [426]:
# honorific 값의 개수를 줄여, 자주 등장하지 않는 값들은 "other"로 묶음
def get_preprocessed_dataset_5(all_df):
    # 주요 호칭 이외의 값들을 "other"로 변환하여 하나로 묶음
    all_df.loc[
        ~((all_df["honorific"] == "Mr") |
          (all_df["honorific"] == "Miss") |
          (all_df["honorific"] == "Mrs") |
          (all_df["honorific"] == "Master")),
        "honorific"] = "other"
    # Embarked(탑승 항구)의 결측치를 "missing"으로 채움
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

In [427]:
# 범주형 변수들을 LabelEncoder를 사용해 수치형으로 변환하는 함수
def get_preprocessed_dataset_6(all_df):
    # 데이터프레임에서 문자열 타입인 컬럼들만 추출
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        # 각 범주형 변수를 LabelEncoder로 수치형으로 변환
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

In [428]:
from torch import nn

# 신경망 모델 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        # 신경망 구조 정의 (은닉층 2개, 각 층 30개의 유닛)
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),  # 입력층 -> 첫 번째 은닉층
            nn.ReLU(),               # 활성화 함수로 ReLU 사용
            nn.Linear(30, 30),       # 첫 번째 은닉층 -> 두 번째 은닉층
            nn.ReLU(),               # ReLU 활성화 함수
            nn.Linear(30, n_output), # 두 번째 은닉층 -> 출력층
        )

    def forward(self, x):
        # 입력 x를 모델에 통과시켜 최종 출력값 반환
        x = self.model(x)
        return x

In [429]:
# 테스트 함수: 테스트 데이터셋을 통해 모델의 예측을 출력
def test(test_data_loader):
    print("[TEST]")
    # 테스트 데이터에서 첫 번째 배치를 가져옴
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    # 테스트를 위한 모델 생성 (입력 노드 11개, 출력 노드 2개)
    my_model = MyModel(n_input=11, n_output=2)
    # 모델을 통해 배치의 입력 데이터를 예측
    output_batch = my_model(batch['input'])
    # 출력의 각 샘플에서 가장 높은 확률의 클래스를 예측값으로 선택
    prediction_batch = torch.argmax(output_batch, dim=1)
    # 예측 결과 출력 (테스트 데이터의 인덱스와 예측값)
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

In [430]:
# 메인 함수
if __name__ == "__main__":
    # 전처리된 데이터셋을 가져와 학습, 검증, 테스트 데이터셋으로 분리
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    # 데이터셋 크기 출력
    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))

    # DataLoader를 사용해 학습, 검증, 테스트 데이터 로드
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 학습 데이터 로드 확인
    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    # 검증 데이터 로드 확인
    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    # 테스트 함수 호출하여 테스트 데이터의 예측 결과 확인
    test(test_data_loader)

C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\928423758.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\377400784.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
[TRAIN]
0 - torch.Size([16, 11]): torch.Size([16])
1 - torch.Size([16, 11]): torch.Size([16])
2 - torch.Size([16, 11]): torch.Size([16])
3 - torch.Size([16, 11]): torch.Size([16])
4 - torch.Size([16, 11]): torch.Size([16])
5 - torch.Size([16, 11]): torch.Size([16])
6 - torch.Size([16, 11]): torch.Size([16])
7 - torch.Size([16, 11]): torch.Size([16])
8 - torch.Size([16, 11]): torch.Size([16])
9 - torch.Size([16, 11]): torch.Size([16])
10 - torch.Size([16, 11]): torch.Size([16])
11 - torch.Size([16, 11]): torch.Size([16])
12 - torch.Size([16, 11]): torch.Size([16])
13 - torch.Size([16, 11]): torch.Size([16])
14 - torch.Size([16, 11]): torch.Size([16])
15 - torch.Size([16, 11]): torch.Size([16])
16 - torch.Size([16, 11]): torch.Size([16])
17 - torch.Size([16, 11]): torch.Size([16])
18 - torch.Size([16, 11]): torch.Size([16])
19 - torch.Size([16, 11]): torch.Size([16])
20 - torch.Size([16, 11]): torch.Size([16])
21 - torc

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd

# 타이타닉 데이터셋 함수 가져오기
train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

def get_data():
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=wandb.config.batch_size)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=wandb.config.batch_size, shuffle=False)
    return train_data_loader, validation_data_loader, test_data_loader

class MyModel(nn.Module):
    def __init__(self, n_input, n_output, activation_fn):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            activation_fn,
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            activation_fn,
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        return self.model(x)

def get_model_and_optimizer(activation_fn):
    model = MyModel(n_input=11, n_output=2, activation_fn=activation_fn)
    optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
    return model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    loss_fn = nn.CrossEntropyLoss()
    n_epochs = wandb.config.epochs
    best_validation_loss = float("inf")

    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss = 0.0
        for train_batch in train_data_loader:
            input, target = train_batch['input'], train_batch['target']
            output = model(input)
            loss = loss_fn(output, target)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_data_loader)

        model.eval()
        validation_loss = 0.0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input, target = validation_batch['input'], validation_batch['target']
                output = model(input)
                loss = loss_fn(output, target)
                validation_loss += loss.item()

        avg_validation_loss = validation_loss / len(validation_data_loader)
        if avg_validation_loss < best_validation_loss:
            best_validation_loss = avg_validation_loss
            best_model_state = model.state_dict()

        # Logging train and validation loss to wandb
        wandb.log({
            "Epoch": epoch,
            "Training Loss": avg_train_loss,
            "Validation Loss": avg_validation_loss
        })

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_validation_loss:.4f}")

    return best_model_state, best_validation_loss

def test_model(model, test_data_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.tolist())

    submission = pd.DataFrame({
        "PassengerId": range(892, 892 + len(predictions)),
        "Survived": predictions
    })
    submission.to_csv("submission.csv", index=False)
    print("submission.csv has been created.")

def main(args):
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="titanic_training",
        name=f"Run_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}",
        config=config
    )

    train_data_loader, validation_data_loader, test_data_loader = get_data()
    activation_functions = {
        "ELU": nn.ELU(),
        "ReLU": nn.ReLU(),
        "LeakyReLU": nn.LeakyReLU(0.01),
        "PReLU": nn.PReLU()
    }

    best_validation_loss = float("inf")
    best_activation_fn = None

    for name, activation_fn in activation_functions.items():
        print(f"Training with {name} activation function.")
        model, optimizer = get_model_and_optimizer(activation_fn)
        model_state, validation_loss = training_loop(model, optimizer, train_data_loader, validation_data_loader)

        print(f"{name} Validation Loss: {validation_loss:.4f}")
        if validation_loss < best_validation_loss:
            best_validation_loss = validation_loss
            best_activation_fn = activation_fn
            best_model_state = model_state

    print(f"Best Activation Function: {best_activation_fn} with Validation Loss: {best_validation_loss:.4f}")

    # 최적의 활성화 함수로 모델 초기화 및 로드 후 테스트 예측
    final_model = MyModel(n_input=11, n_output=2, activation_fn=best_activation_fn)
    final_model.load_state_dict(best_model_state)
    test_model(final_model, test_data_loader)
    wandb.finish()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--wandb", action=argparse.BooleanOptionalAction, default=True, help="Use wandb logging")
    parser.add_argument("-b", "--batch_size", type=int, default=100, help="Batch size (int, default: 100)")
    parser.add_argument("-e", "--epochs", type=int, default=1000, help="Number of training epochs (int, default: 1000)")
    args = parser.parse_args(args=[])
    main(args)


C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\928423758.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\377400784.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

Training with ELU activation function.
Epoch 100, Training Loss: 0.4142, Validation Loss: 0.4279
Epoch 200, Training Loss: 0.3989, Validation Loss: 0.4163
Epoch 300, Training Loss: 0.3689, Validation Loss: 0.4091


![결과](https://github.com/MinSeokCSE/Deep/blob/main/kaggle.png?raw=true) 